In [1]:
import numpy as np
import random

# select {col_name} from table_name where col_name <=> constant
# insert values(tuple) into table_name

class simpleSQL:
    def __init__(self) -> None:
        self.token=[]
        
    # def __init__(self,token = None) -> None:
    #     self.token=token
        
    def add(self,x):
        self.token.append(x)
        
    def toStr(self):
        for i in range(len(self.token)):
            print(str(self.token[i].value)+" ",end="")
        print()

class key:
    def __init__(self,value,type) -> None:
        self.value=value
        self.type=type
        # self.name=name


In [2]:
class foreign_constraint:
    def __init__(self,tb1,col1,tb2,col2) -> None:
        self.tb1=tb1
        self.col1=col1
        self.tb2=tb2
        self.col2=col2

In [3]:
class Table:
    def __init__(self,tb_name,col) -> None:
        self.name=tb_name
        self.col=col
        self.col_data_dis={}
    
    def addCharacteristics(self,col_name,data_dis):
        col_name_set=set(self.col[0:len(self.col)])
        if col_name not in col_name_set:
            print("error: add data characteristics failed. Col name not found.")
        else:
            self.col_data_dis[col_name]=data_dis

In [4]:
class DBschema:
    def __init__(self,tbs,cons=None) -> None:
        self.tables=tbs
        # self.tb_num=len(tbs)
        self.constraint=cons

In [5]:
# Random sampling in a given probability distribution graph
# ---- return the position(i.e. l+index) corresponding to the sample num.
def randNumGen(l,r,pdg):
    if r-l+1 != len(pdg):
        print("l is "+str(l)+" r is "+str(r)+" and pdg is "+str(pdg))
        print("error : pdg not match [l,r]")
        return l-1
    one_=sum(pdg)
    if abs(one_-1.0)>1e-5:
        print("error : probability sum "+str(one_)+" do not equal 1")
        return l-1
    pdg_=np.array(pdg)
    pdg_=[round(i*1000) for i in pdg_]
    maxv=sum(pdg_)
    num=random.randint(0,maxv-1)
    index=0
    for i in range(len(pdg_)):
        num=num-pdg_[i]
        if num<0:
            index=i
            break
    return l+index

In [6]:
dict={}
for i in range(1,6):
    dict[i]=0
for i in range(1,100000,1):
    # print(randNumGen(1,5,np.full((5),0.2)))
    # dict[randNumGen(1,5,np.full((5),0.2))]+=1
    dict[randNumGen(1,5,[0.1,0.2,0.3,0.4,0.0])]+=1
for i in range(1,6):
    print(dict[i])

10043
20013
30057
39886
0


In [7]:
dict={}
dict[0]=0
dict[1]=0
for i in range(1000):
    dict[randNumGen(0,1,np.array([0.7,0.3]))]+=1
print(dict[0])
print(dict[1])

716
284


In [196]:
#def getNTable(dbs,n):
#    all_tb=[dbs.tables[i] for i in len(dbs.tables)]
#    if len(dbs.tables)<=n

In [197]:
# data access distribution
class feature:
    def __init__(self,rwRatio,avtb) -> None:
        self.rwRatio=rwRatio
        self.avtb=avtb
        

In [198]:
class SQLGen:
    def __init__(self,dbs) -> None:
        self.sqlSet=[]
        self.sql=simpleSQL()
        self.dbs=dbs

    def generate(self,condition):
        rw_choice=randNumGen(0,1,np.array([condition.rwRatio,1-condition.rwRatio]))
        tb_num=randNumGen(1,2,[0.5,0.5])
        tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,[1.0/len(self.dbs.tables) for i in range(len(self.dbs.tables))])]
        cons=randNumGen(0,3,[1.0/4 for i in range(4)])
        data=randNumGen(0,99,[1.0/100 for i in range(100)])
        if rw_choice==0:
            self.sql.add(key(value="select",type="keyword"))
            self.sql.add(key(value="*",type="colname"))
            self.sql.add(key(value="from",type="keyword"))
            self.sql.add(key(value=tb_choice.name,type="tbname"))
            self.sql.add(key(value="where",type="keyword"))
            self.sql.add(key(value=tb_choice.col[0].value,type="colname"))
            if cons==0:
                self.sql.add(key(value=">",type="compare"))
            else:
                if cons==1:
                    self.sql.add(key(value="<",type="compare"))
                else:
                    if cons==2:
                        self.sql.add(key(value="=",type="compare"))
                    else: 
                        if cons==3:
                            self.sql.add(key(value="!=",type="compare"))
                        else: 
                            pass
            self.sql.add(key(value=data,type="value"))
            self.sql.add(key(value=";",type="end"))
        else:
            self.sql.add(key(value="insert",type="keyword"))
            self.sql.add(key(value="into",type="keyword"))
            self.sql.add(key(value=tb_choice.name,type="tbname"))
            self.sql.add(key(value="values",type="keyword"))
            tmp="("
            for i in range(len(tb_choice.col)):
                if i!=0:
                    tmp+=","
                tmp+=str(randNumGen(0,99,[1.0/100 for i in range(100)]));
            tmp+=")"
            self.sql.add(key(value=tmp,type="value"))
            self.sql.add(key(value=";",type="end"))
        
        self.sqlSet.append(self.sql)
        return self.sql


In [199]:
tb=Table("lzz",[key("age",int),key("weight",int)])
dbs=DBschema(tbs=[tb],cons=None)
sg=SQLGen(dbs=dbs)
fea=feature(0.7,1)
sg.generate(condition=fea)
sg.sql.toStr()

insert into lzz values (64,80) ; 


In [200]:
#
#

7
3
